## Ingesting PDF

In [12]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"
import os
from langchain_community.document_loaders import PDFMinerLoader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\pypdfium2.exe' -> 'c:\\Python311\\Scripts\\pypdfium2.exe.deleteme'



In [13]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader

In [14]:
documents = []
    # Loop through all the files in the "docs" directory
for root, dirs, files in os.walk("docs"):
        for file in files:
            if file.endswith(".pdf"):
                print(file)
                loader = PDFMinerLoader(os.path.join(root, file))
                data=loader.load()  # Add the loaded documents to the list

WEF_The_Global_Cooperation_Barometer_2024.pdf


In [10]:
# Preview first page
data[0].page_content

'In collaboration with \nMcKinsey & Company\n\nThe Global Cooperation \nBarometer 2024\n\nI N S I G H T   R E P O R T\n\nJ A N U A R Y   2 0 2 4\n\n\x0cContents\n\nForeword \n\nAbout the Global Cooperation Barometer\n\nExecutive summary \n\nIntroduction: The state of global cooperation\n\nFive pillars of global cooperation\n\nPillar 1 Trade and capital \n\nPillar 2 Innovation and technology\n\nPillar 3 Climate and natural capital\n\nPillar 4 Health and wellness\n\nPillar 5 Peace and security\n\nConclusion: Towards a more cooperative future \n\nAppendix: Sources and methodology\n\nContributors\n\nEndnotes\n\n3\n\n4\n\n6\n\n7\n\n9\n\n9\n\n11\n\n13\n\n15\n\n17\n\n19\n\n20\n\n22\n\n23\n\nDisclaimer \nThis document is published by the  \nWorld Economic Forum as a contribution \nto a project, insight area or interaction. \nThe findings, interpretations and \nconclusions expressed herein are a result \nof a collaborative process facilitated and \nendorsed by the World Economic Forum \nbut who

## Vector Embeddings

In [7]:
!ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 970aa74c0a90...  86% ▕█████████████   ▏ 237 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  86% ▕█████████████   ▏ 237 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  86% ▕█████████████   ▏ 237 MB/274 MB                  pulling manifest 
pulling 970aa74c0a90...  86% ▕█████████████   ▏ 237 MB/274 MB                  pulling manifest 
pulli

In [20]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
mistral:latest         	61e88e884507	4.1 GB	12 seconds ago	
nomic-embed-text:latest	0a109f422b47	274 MB	13 minutes ago	


In [15]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [16]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [17]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 11/11 [01:28<00:00,  8.00s/it]


## Retrieval

In [18]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [21]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [22]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [23]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [25]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [26]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


' The PDF document is the Global Cooperation Barometer 2024 report published by the World Economic Forum (WEF). It provides an analysis of global cooperation in various areas, including health, economy, and peace and security. The report discusses the state of cooperation prior to the COVID-19 pandemic, the surge in cooperation during the pandemic response, and the current trends and challenges in each area. It also highlights the need for continued cooperation to address ongoing issues and future crises.'

In [ ]:
# Delete all collections in the db
vector_db.delete_collection()